In [4]:
import folium
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np

# Load map data and housing price data
merged_data = pd.read_excel('C:/Users/ZJH/Desktop/datatset/results_df.xlsx')
geo_df = gpd.read_file('C:/Users/ZJH/Desktop/social data/final/capital_region_municipalities.geojson')
house_prices = pd.read_excel('C:/Users/ZJH/Desktop/datatset/house_price.xlsx')
house_prices.set_index('Municipality', inplace=True)

# Initialize weights and sliders
weights = {
    'eat': 0,
    'gym': 0,
    'hospital': 0,
    'park': 0,
    'school': 0,
    'shop': 0,
    'station': 0,
    'company':0,
}
sliders = {fac: widgets.FloatSlider(value=w, min=0, max=1, step=0.01, description=f'{fac.capitalize()}:') for fac, w in weights.items()}
total_weight_label = widgets.Label('Total Weight: 0.00')

# Function to update total weight and enforce constraints
def update_total_weight(change):
    total_weight = sum(slider.value for slider in sliders.values())
    total_weight_label.value = f"Total Weight: {total_weight:.2f}"
    start_button.disabled = not np.isclose(total_weight, 1.0)

for slider in sliders.values():
    slider.observe(update_total_weight, 'value')

# Create the output containers
map_container = widgets.Output()
chart_container = widgets.Output()

def create_map():
    merged_df = geo_df.merge(merged_data, how='left', on='name_gn_spell_spellofna_text')
    for fac, weight in weights.items():
        if fac in merged_df.columns:
            merged_df[fac + '_score'] = merged_df[fac] * weight
    merged_df['total_score'] = merged_df[[f"{fac}_score" for fac in weights if f"{fac}_score" in merged_df.columns]].sum(axis=1)
    # 计算Z-得分
    mean_score = merged_df['total_score'].mean()
    std_dev = merged_df['total_score'].std()
    merged_df['z_score'] = (merged_df['total_score'] - mean_score) / std_dev

    # 将Z-得分转换为0-100百分制
    merged_df['normalized_score'] = round(50 + (merged_df['z_score'] * 10))  # 这里*10并加50是一种常见的转换方式

    # 处理超出0-100范围的情况
    merged_df['normalized_score'] = merged_df['normalized_score'].clip(lower=0, upper=100)
    merged_df['rank'] = merged_df['normalized_score'].rank(ascending=False, method='min')
    # Define color by rank
    def get_color(rank):
        if rank == 1:
            return '#1E90FF'  # Darker blue for the highest score
        elif rank == 2:
            return '#87CEFA'  # Medium blue for the second-highest score
        elif rank == 3:
            return '#B0E0E6'  # Lighter blue for the third-highest score
        else:
            return '#CCCCCC'  # Grey for others

    m = folium.Map(location=[55.6761, 12.5683], zoom_start=10)
    folium.GeoJson(
        merged_df,
        style_function=lambda feature: {
            'fillColor': get_color(feature['properties']['rank']),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=folium.GeoJsonTooltip(
            fields=['name_gn_spell_spellofna_text','normalized_score', 'rank'],
            aliases=['Municipality:', 'Score:', 'Rank:']
        )
    ).add_to(m)
    with map_container:
        clear_output(wait=True)
        display(m)
    return merged_df


def create_chart(municipalities):
    plt.figure(figsize=(10, 6))
    for municipality in municipalities:
        plt.plot(house_prices.columns, house_prices.loc[municipality], marker='o', linestyle='-', label=f'{municipality}')
    plt.title('Yearly Price Comparison for Top 3 Municipalities')
    plt.xlabel('Year')
    plt.ylabel('Price')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend(title='Municipality')
    plt.tight_layout()
    with chart_container:
        clear_output(wait=True)
        plt.show()

# "Start Update" button functionality
start_button = widgets.Button(description="Start Update", disabled=True)

def on_button_clicked(b):
    for slider in sliders.values():
        weights[slider.description.strip(':').lower()] = slider.value
    merged_df = create_map()
    top_municipalities = merged_df.nsmallest(3, 'rank')['name_gn_spell_spellofna_text'].tolist()
    create_chart(top_municipalities)

start_button.on_click(on_button_clicked)

# Layout the widgets and output
control_box = widgets.VBox([total_weight_label] + list(sliders.values()) + [start_button])
layout = widgets.HBox([control_box, widgets.VBox([chart_container, map_container])])
display(layout)
